### Call the model serving endpoint
**Relevant docs**:
 - [Route optimized endpoints](https://docs.databricks.com/gcp/en/machine-learning/model-serving/route-optimization)
 - [Databricks Python SDK](https://databricks-sdk-py.readthedocs.io/en/latest/index.html)
 - [Databricks Python SDK serving_endpoint_data_plane](https://databricks-sdk-py.readthedocs.io/en/latest/workspace/serving/serving_endpoints_data_plane.html)
 - [Inference tables](https://docs.databricks.com/aws/en/ai-gateway/inference-tables)
 - [Load testing for model serving endpoints](https://docs.databricks.com/aws/en/machine-learning/model-serving/what-is-load-test)
 - [Configure a load test for model serving endpoints](https://docs.databricks.com/aws/en/machine-learning/model-serving/configure-load-test)
 - [Databricks secrets](https://docs.databricks.com/gcp/en/security/secrets/)
 - [Databricks CLI](https://docs.databricks.com/aws/en/dev-tools/cli/)

In [0]:
%pip install databricks-sdk
%restart_python

In [0]:
from databricks.sdk import WorkspaceClient

In [0]:
dbutils.widgets.text('catalog_name','','Enter catalog name')
dbutils.widgets.text('schema_name','','Enter schema name')

In [0]:
catalog_name = dbutils.widgets.get('catalog_name')
schema_name = dbutils.widgets.get('schema_name')
table_name = "advanced_churn_bronze_customers"

uc_location = f"{catalog_name}.{schema_name}.{table_name}"
print(f"UC location: {uc_location}")

In [0]:
# Get a temporary Oauth token from the model serving endpoint. Oauth tokens are required for authentication when using route optimized endpoints, which provide lower latency
token = ""

In [0]:
import os
import requests
import numpy as np
import pandas as pd
import json

def create_tf_serving_json(data):
    return {'inputs': {name: data[name].tolist() for name in data.keys()} if isinstance(data, dict) else data.tolist()}

url = ""
def score_model(dataset):
    url = url
    headers = {'Authorization': f'Bearer {token}', 'Content-Type': 'application/json'}
    ds_dict = {'dataframe_split': dataset.to_dict(orient='split')} if isinstance(dataset, pd.DataFrame) else create_tf_serving_json(dataset)
    data_json = json.dumps(ds_dict, allow_nan=True)
    response = requests.request(method='POST', headers=headers, url=url, data=data_json)
    if response.status_code != 200:
        raise Exception(f'Request failed with status {response.status_code}, {response.text}')
    return response.json()

In [0]:
df = spark.table(uc_location).drop("customer_i_d", "churn").toPandas()
df.head()

In [0]:
score_model(df[:3])

####Query the endpoint using the Workspaces Python SDK (preferred)
 - Automatically refreshes Oauth tokens
 - Provides retries and exponential backoff


In [0]:
# Authenticate using service principal credentials saved as Databricks secrets
CLIENT_ID = dbutils.secrets.get(scope="scope_name", key=f"secret_key")
CLIENT_SECRET = dbutils.secrets.get(scope="scope_name", key=f"secret_key")
DATABRICKS_HOST = "" #Enter the databricks workspace_url

In [0]:
ds_dict = {'dataframe_split': df[:3].to_dict(orient='split')}
ds_dict

In [0]:
w = WorkspaceClient(
  host          = DATABRICKS_HOST,
  client_id     = CLIENT_ID,
  client_secret = CLIENT_SECRET)

predictions = w.serving_endpoints_data_plane.query(name = "basic_model_endpoint",
                     dataframe_records = df[:3].to_dict(orient='records')
                     )

predictions.predictions

####View inference table results

In [0]:
inference_table = spark.table("shared.mlc_schema.basic_model_endpoint_payload")
display(inference_table)